# How to stream events from within a tool

If your LangGraph graph needs to use tools that call LLMs (or any other LangChain `Runnable` objects -- other graphs, LCEL chains, retrievers, etc.), you might want to stream events from the underlying `Runnable`. This guide shows how you can do that.

## Setup

In [1]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai

In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

OPENAI_API_KEY:  ········


## Define graph and tools

We'll use a simple ReAct agent for this guide

In [3]:
from langchain_core.callbacks import Callbacks
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool

from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

In [4]:
@tool
async def get_items(place: str, callbacks: Callbacks) -> str:  # <--- Accept callbacks
    """Use this tool to look up which items are in the given place."""
    template = ChatPromptTemplate.from_messages(
        [
            (
                "human",
                "Can you tell me what kind of items i might find in the following place: '{place}'. "
                "List at least 3 such items separating them by a comma. And include a brief description of each item..",
            )
        ]
    )
    chain = template | llm.with_config(
        {
            "run_name": "Get Items LLM",
            "tags": ["tool_llm"],
            "callbacks": callbacks,  # <-- Propagate callbacks
        }
    )
    chunks = [chunk async for chunk in chain.astream({"place": place})]
    return "".join(chunk.content for chunk in chunks)

In [5]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo")
tools = [get_items]
agent = create_react_agent(llm, tools=tools)

## Stream events from the graph

In [6]:
async for event in agent.astream_events({"messages": [("human", "what items are on the shelf?")]}, version="v2"):
    tags = event.get("tags", [])
    if event["event"] == "on_chat_model_stream" and "tool_llm" in tags:
        print(event["data"]["chunk"].content, end="", flush=True)

/Users/vadymbarda/.virtualenvs/langgraph/lib/python3.12/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


1. Books - These are typically rectangular objects made of paper bound together, containing written or printed material such as stories, information, or pictures. They are commonly found on shelves in libraries, bookstores, or personal collections.

2. Photo frames - These are decorative items used to display photographs or artwork. They come in various sizes, materials, and designs and are commonly placed on shelves to showcase special memories or moments.

3. Decorative figurines - These are small sculptures or statues used for ornamental purposes. They can be made of various materials such as ceramic, glass, or metal, and are often placed on shelves to add a personal touch to the decor of a room.

Let's inspect the last event to get the final list of messages from the agent

In [7]:
final_messages = event["data"]["output"]["messages"]

In [8]:
for message in final_messages:
    message.pretty_print()

================================ Human Message =================================

what items are on the shelf?
================================== Ai Message ==================================
Tool Calls:
  get_items (call_hKfYnR80GEZD3xdXnIMsXI0Y)
 Call ID: call_hKfYnR80GEZD3xdXnIMsXI0Y
  Args:
    place: shelf
================================= Tool Message =================================
Name: get_items

1. Books - These are typically rectangular objects made of paper bound together, containing written or printed material such as stories, information, or pictures. They are commonly found on shelves in libraries, bookstores, or personal collections.

2. Photo frames - These are decorative items used to display photographs or artwork. They come in various sizes, materials, and designs and are commonly placed on shelves to showcase special memories or moments.

3. Decorative figurines - These are small sculptures or statues used for ornamental purposes. They can be made of various mate

You can see that the content of the `ToolMessage` is the same as the output we streamed above